# Combinatorial Prediction Market

2010年以降、Automated Market Makerの一般化が進んでいるが、[それ以前になされた考察](http://blog.oddhead.com/2008/12/22/what-is-and-what-good-is-a-combinatorial-prediction-market/)では、個別事例の列挙が中心だった。

1. Boolean Betting
 * 例えば州選挙で2政党のどちらが勝利するかを賭けの対象とした場合、$2^{50}$の組み合わせがある。
 * [Hansonによる、Combinatorial Information Market Design](http://mason.gmu.edu/~rhanson/combobet.pdf)では、そのへんの考察を進めている。
2. Tournament Betting
3. Permutation betting ...  例えば競馬


## [Bayesian Market Maker](http://www.cse.wustl.edu/~allenlavoie/papers/bmm.pdf)

### Information-Based Market Making

[Glosten Milgrom Model](https://pdfs.semanticscholar.org/5827/ca4a5ac97e717fb5768f313079e813cebe86.pdf)というものに基づく。
経済学ではすごく有名らしい、1985年の論文

時刻$t$で、マーケットメイカーは証券価格の事前分布$p_t(v)$を持つ。
トレーダーはシグナル$s$を受け取る。$s$の分散はトレーダーの情報の不確実さを表す。

マーケットメイカーはbidとaskの値段を提示し、$s < bid$ならばトレーダーは売り、$s > ask$ならば買う。

マーケットメイカーは均衡を達成するため、以下かの等式を解いてaskとbidの価格を調整する。

$$
ask = E_{p_t(v)}[v|s > ask]: bid = E_{p_t(v)}[v|s < bid]
$$

これが過去のbayesian market maker（例えば[A Learning Market-Maker in the Glosten-Milgrom Model](https://pdfs.semanticscholar.org/768d/c8c25f7b621e06648b458e51cbaa0a8ee8c6.pdf)）の礎になっている。

その後、トレーダーの行動に応じて、価格を更新していく

強化学習の言葉で言うと、bidとaskの提示はactionに、トレーダーの行動は環境からの入力であると言える。
大抵の強化学習の状況と同様、actionが、1. 情報の開示　2. 報酬の獲得
の2つを行う。

上記のGlosten-Milgrom Modelの論文では、zero profit、あるいはprofit maximizingのためのアップデートの両方を検討している。

単純化のために、マーケットメイカーの事前分布$V$と、トレーダーへの情報$s$はともに正規分布を仮定している。

[Adapting to a Market Shock: Optimal Sequential Market-Making](http://papers.nips.cc/paper/3600-adapting-to-a-market-shock-optimal-sequential-market-making.pdf)
で紹介されたQ学習を用いたマーケットメイキングについて簡単に説明する。

マーケットメイカーの情報Disadvantage(二者間での情報の不確実の比率)を$\rho_t = \sigma_t/\sigma_{\eta}$(それぞれの$\sigma$は正規分布の分散を表す)とする。

$Q(\rho)$関数が非常に重要な弱割を果たす。以下のようにしてaskの値を計算する。
$$
ask = \mu_t + \sigma_{\eta}Q(\rho_t) \sqrt{1+\rho^2_t}
$$
トレーダーが$z^- < s < z^+$の間にあるシグナルを提示するとする、その時マーケットメイカー側は以下のようにして正規分布を更新する。

$\mu_{t+1} = \mu_t + \sigma_t * \frac{B}{A}$ ... 平均

$\sigma^2_{t+a} = \sigma^2_t \bigl( 1-\frac{AC+B^2}{A^2} \bigr)$ ... 分散

ただし、A,B,Cは$z^-, z^+, \mu_t, \rho_t, \sigma_{eta}$の関数であり、詳細は上記の論文に乗っている。

このアルゴリズム(ZP)はfized trade sizeのみquoteできる。任意のshareをquoteできなくては実用的ではない。そこで、$\alpha$という固定のトレードサイズで運営できるようなヒューリスティックを導入する。

$q_t = \mu_t$をスポット価格とし、$Q$をトレーダーが購入したいshareとする。
したがって$[Q/\alpha]$の独立な